In [1]:
import os

In [2]:
os.chdir("../")

In [3]:
%pwd

'h:\\Text Summarizer\\Text-Summarization'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path
    load_data_dir : Path

In [5]:
from TextSummarizer.constants import *
from TextSummarizer.utils.common import read_yaml, create_directories

In [6]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = Path("config/config.yaml"),
        params_filepath = Path("params.yaml")):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir,
            load_data_dir=config.load_data_dir
        )

        return data_ingestion_config

In [7]:
import urllib.request as request
import zipfile
from TextSummarizer.logging import logger
from TextSummarizer.utils.common import get_size
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import pandas as pd


In [8]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"Downloaded file: {filename}")
        else:
            logger.info(f"File already exists: {self.config.local_data_file}")
        
    
    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)


    def clean_text(self, text):
        text = text.lower()
        text = re.sub(r'[^\w\s]', '', text)

        stop_words = set(stopwords.words('english'))
        words = text.split()
        words = [word for word in words if word not in stop_words]
        text = ' '.join(words)

        lemmatizer = WordNetLemmatizer()
        words = [lemmatizer.lemmatize(word) for word in words]
        text = ' '.join(words)

        return text
    
    def merge(self):

        categories = ['sport', 'business', 'entertainment', 'politics', 'tech']  
        articles_dir = f'{self.config.load_data_dir}/News Articles'
        summaries_dir = f'{self.config.load_data_dir}/Summaries'

        data = []

        for category in categories:
            article_files = os.listdir(os.path.join(articles_dir, category))
            for file in article_files:
                with open(os.path.join(articles_dir, category, file), 'r', encoding='latin-1') as f_article, \
                    open(os.path.join(summaries_dir, category, file), 'r', encoding='latin-1') as f_summary:
                    data.append({'text': f_article.read(), 'summary': f_summary.read()})

        df = pd.DataFrame(data)
        df['text'] = df['text'].apply(self.clean_text)
        df['summary'] = df['summary'].apply(self.clean_text)
        df.to_csv(os.path.join(self.config.root_dir, 'dataset.csv'), index=False)
        df.head()

In [10]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
    data_ingestion.merge()
except Exception as e:
    raise e

[2024-05-04 15:53:45,619: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-05-04 15:53:45,623: INFO: common: yaml file: params.yaml loaded successfully]
[2024-05-04 15:53:45,625: INFO: common: created directory at: artifacts]
[2024-05-04 15:53:45,627: INFO: common: created directory at: artifacts/data_ingestion]


[2024-05-04 15:53:49,551: INFO: 2255856556: Downloaded file: artifacts/data_ingestion/BBCNewsSummary.zip]
